In [ ]:
!pip install wheel setuptools pip --upgrade
!pip install --upgrade openai

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.0 MB/s eta 0:00:00
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
import pandas as pd
import random
from openai import OpenAI
import time
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

census_df = pd.read_csv('census_small.csv', header = None)

# Define the column names
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
                'occupation', 'relationship', 'race', 'sex', 'capital-gain',
                'capital-loss', 'hours-per-week', 'native-country', 'income']

# Assign the column names to the DataFrame
census_df.columns = column_names
census_df.fillna('Unknown')

print( census_df['income'].unique() )
census_df['income'] = census_df['income'].replace(' <=50K', 'not exceeding')
census_df['income'] = census_df['income'].replace(' >50K', 'exceeding')
census_df.drop(columns=['fnlwgt', 'education-num'], inplace=True)
print( census_df['income'].unique() )

# # Apply qcut to each numerical column to divide into 20 bins
# for column in census_df.select_dtypes(include=[np.number]).columns:
#     census_df[column], bins = pd.cut(census_df[column], 30, duplicates='drop', retbins=True)

# create a tuning set
tuning_set = census_df.sample(n=800, random_state=0)
percentage_yes = (tuning_set['income'] == 'exceeding').mean()
print(percentage_yes)

row_series = tuning_set.iloc[0][:-1]
target = tuning_set.iloc[0]['income']

all_features = ''
for feature, value in row_series.items():
    all_features += feature + ', '

print(all_features)


selected_rows_1 = census_df.loc[census_df['income'] == 'exceeding'].head(5)

selected_rows_2 = census_df.loc[census_df['income'] == 'not exceeding'].head(4)

# Combine the two DataFrames into a new DataFrame
combined_df = pd.concat([selected_rows_1, selected_rows_2], ignore_index=True)

samples = []
targets = []
for i in range( len(combined_df) ):
    row_series = combined_df.iloc[i][:-1]
    target = combined_df.iloc[i]['income']

    overall_feature_levels = ''
    for feature, value in row_series.items():
        feature_level = (f"Feature: {feature}, Value: {value}")
        overall_feature_levels += feature_level + '; '

    samples.append(overall_feature_levels)
    targets.append(target)

print(samples)
print(targets)

[' <=50K' ' >50K']
['not exceeding' 'exceeding']
0.265
age, workclass, education, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country, 
['Feature: age, Value: 52; Feature: workclass, Value:  Self-emp-not-inc; Feature: education, Value:  HS-grad; Feature: marital-status, Value:  Married-civ-spouse; Feature: occupation, Value:  Exec-managerial; Feature: relationship, Value:  Husband; Feature: race, Value:  White; Feature: sex, Value:  Male; Feature: capital-gain, Value: 0; Feature: capital-loss, Value: 0; Feature: hours-per-week, Value: 45; Feature: native-country, Value:  United-States; ', 'Feature: age, Value: 31; Feature: workclass, Value:  Private; Feature: education, Value:  Masters; Feature: marital-status, Value:  Never-married; Feature: occupation, Value:  Prof-specialty; Feature: relationship, Value:  Not-in-family; Feature: race, Value:  White; Feature: sex, Value:  Female; Feature: capital-gain, Value: 14084; Feature:

In [ ]:
API_KEY = ''
client = OpenAI(api_key = API_KEY)
model_id = 'gpt-3.5-turbo-0125'

APE_prompt = f"""
You are given a set of feature-value pairs that represents a person's profile. Your task is to propose a creative, detailed, and step-by-step algorithm to reformulate and enrich this profile. The goal is of the algorithm is to perform a thorough data engineering among the features, so that it is easier for a LLM to distinguish whether the person has a annual income that exceeds $50,000, specifically in the year of 1995. Below are some sample profiles as refrences.

Examples:
- Job Description: '{samples[0]}'; Is the job Fraud or not: '{targets[0]}'
- Job Description: '{samples[1]}'; Is the job Fraud or not: '{targets[1]}'
- Job Description: '{samples[2]}'; Is the job Fraud or not: '{targets[2]}'
- Job Description: '{samples[6]}'; Is the job Fraud or not: '{targets[6]}'
- Job Description: '{samples[7]}'; Is the job Fraud or not: '{targets[7]}'
- Job Description: '{samples[8]}'; Is the job Fraud or not: '{targets[8]}'

For each step of the algorithm, number and then start it on a new line; you must start each step with '###------- Step 1: ' The proposed algorithm will later be submitted to a LLM for processing.
Important: Do NOT refer to any external database; Do NOT perform the item counting. Do NOT perform normalization. Do NOT perform vector generations. Do NOT perform similarity checking. Do NOT propose how to train or generate a recommendation system. ONLY propose things that a LLM can do on its own!
"""

prompts = []
prompt_performances = {}
for i in range(7):

    completion = client.chat.completions.create(
        model = model_id, temperature = 1.0,

        messages=[{"role": "system", "content": "Please come up with a step-by-step, very detailed, clear, and novel algorithm for reformulating and enriching a person's profile. Return the steps ONLY!!! NO more than 5 steps. You MUST NOT directly include whether the person has a annual income that exceeds $50,000!!!"},
                    {"role": "user", "content": APE_prompt}],
        timeout = 1200)

    candidate_prompt = completion.choices[0].message.content
    prompts.append(candidate_prompt)
    print(candidate_prompt)
    print()


In [ ]:
import re

# Regular expression to split the text into individual algorithms
algorithm_pattern = r"###------- Step \d: [\w\s]+"

# Split the text based on the pattern
split_text = re.split(algorithm_pattern, candidate_prompt)

# Extract the algorithm headers (for identification)
headers = re.findall(algorithm_pattern, candidate_prompt)

# Removing the first empty string from the split if exists (because of the split at the start)
split_text = [t.strip() for t in split_text if t.strip()]

# Create a dictionary where each algorithm is stored separately
algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

# Display each algorithm separately
steps = []
for header, content in algorithms.items():
    steps.append( [header, content] )
    print(f"{header}:\n{content}\n")


In [ ]:
system_msg = "Please serve as a binary classifier on user income level."

prompts = []
prompts.append("""###------- Step 1: Identify key feature-value pairs such as age, education, occupation, workclass, and marital status that may correlate with an annual income exceeding $50,000 in the year 1995.
###------- Step 2: Convert categorical variables into numerical representations, for example, mapping education levels to numerical values or encoding occupation types.
###------- Step 3: Calculate derived features such as the total capital (capital-gain minus capital-loss) and years of education based on the provided education level.
###------- Step 4: Analyze the relationship between hours-per-week and income level to identify any potential trends or thresholds that may indicate high-income levels.
###------- Step 5: Iterate through the enriched profile data to identify patterns or combinations of features that are common among individuals with an annual income exceeding $50,000 in 1995.
""")

for candidate_prompt in prompts:
    print(candidate_prompt)
    print()

    # Regular expression to split the text into individual algorithms
    algorithm_pattern = r"###------- Step \d: [\w\s]+"

    # Split the text based on the pattern
    split_text = re.split(algorithm_pattern, candidate_prompt)

    # Extract the algorithm headers (for identification)
    headers = re.findall(algorithm_pattern, candidate_prompt)

    # Removing the first empty string from the split if exists (because of the split at the start)
    split_text = [t.strip() for t in split_text if t.strip()]

    # Create a dictionary where each algorithm is stored separately
    algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

    # Display each algorithm separately
    steps = []
    for header, content in algorithms.items():
        steps.append( [header, content] )
        print(f"{header}:\n{content}\n")


    right_count_pos = 0
    right_count_neg = 0
    compressed_right_count_pos = 0
    compressed_right_count_neg = 0
    total_pos = 0
    total_neg = 0
    total = 0

    # you may also set range smaller for a subset
    for i in range( len(tuning_set) ):
        row_series = tuning_set.iloc[i][:-1]
        target = tuning_set.iloc[i]['income']

        overall_feature_levels = ''
        for feature, value in row_series.items():
            feature_level = (f"Feature: {feature}, Value: {value}")
            overall_feature_levels += feature_level + '; '


        original_prompt = (
            "You are given an user description with each feature-value pair presented."
            "Please determine whether the user had an income higher than $50k in the year of 1993. First write a step-by-step reasoning paragraph, and then output 'exceeding' or 'not exceeding' strictly, in lower case:"
            "Below is the user description: \n\n"
            f"{overall_feature_levels}"
        )

        # original_prompt = (
        #     f"Given the following user description: \n\n {overall_feature_levels};\n\n"
        #     f"Please determine whether the user had an income higher than $50k in the year of 1993. Do NOT explain anything, just output 'exceeding' or 'not exceeding', in lower case:"
        # )

        completion = client.chat.completions.create(
                model = model_id, temperature = 0, seed = 0,
                messages=[{"role": "system", "content": system_msg},
                            {"role": "user", "content": original_prompt }],
                timeout = 1200)

        original_pred = completion.choices[0].message.content
        # print(original_pred)
        # print()

        total += 1
        if target == 'exceeding':
            total_pos += 1
        else:
            total_neg += 1

        if target in original_pred:
            if (target == 'exceeding'):
                right_count_pos += 1
            if (target == 'not exceeding'):
                right_count_neg += 1



        overall_with_steps = "Original person's profile: " + overall_feature_levels + '\n\n'
        for i in range(len(algorithms)):

            reformulation_prompt = f"Please thoroughly reformulate the person's profile based on the following instruction:\n\n{steps[i][0]}\n{steps[i][1]}\n\nprofile to reformulate: {overall_with_steps}\n\nYou MUST return the reformulation of the profile ONLY!!!"

            completion = client.chat.completions.create(
                    model = model_id, temperature = 0.0, max_tokens = 512,

                    messages=[{"role": "system", "content": "Please reformulate the person's profile to be extremely informative and detailed for LLM to interpret better. You are allowed to fill in unspecified information based on your domain expertise!"},
                                {"role": "user", "content": reformulation_prompt}],
                    timeout = 1200)

            reformulated_history = completion.choices[0].message.content
            # print('Step ' + str(i) + ' Results: ')
            # print(reformulated_history)
            overall_with_steps += steps[i][0] + '\n' + steps[i][1] + reformulated_history + '\n\n'
            # print()
            # print()

        print(overall_with_steps)
        print()


        census_prompt = (
            "You are given an user description with each feature-value pair presented."
            "Please determine whether the user had an income higher than $50k in the year of 1993. First write a step-by-step reasoning paragraph, and then output 'exceeding' or 'not exceeding' strictly, in lower case:"
            "Below is the user description: \n\n"
            f"{overall_with_steps}"
        )

        # census_prompt = (
        #     f"Given the following user description: \n\n {overall_with_steps};\n\n"
        #     f"Please determine whether the user had an income higher than $50k in the year of 1993. Do NOT explain anything, just output 'exceeding' or 'not exceeding', in lower case:"
        # )

        compressed_completion = client.chat.completions.create(
                model = model_id, temperature = 0, seed = 0,
                messages=[{"role": "system", "content": system_msg},
                            {"role": "user", "content": census_prompt}],
                timeout = 1200)

        compressed_pred = compressed_completion.choices[0].message.content
        # end


        if target in compressed_pred:
            if (target == 'exceeding'):
                compressed_right_count_pos += 1
            if (target == 'not exceeding'):
                compressed_right_count_neg += 1


        # print(converted_feature_levels)
        print(right_count_pos, right_count_neg)
        print(compressed_right_count_pos, compressed_right_count_neg)
        print()

        if total % 100 == 0 or total == census_df.shape[0]:
            accuracy = (right_count_pos/total_pos) * 0.5 + (right_count_neg/total_neg) * 0.5
            compressed_accuracy = (compressed_right_count_pos/total_pos) * 0.5 + (compressed_right_count_neg/total_neg) * 0.5
            print('Accuracy: ', accuracy)
            print('Compressed Accuracy: ', compressed_accuracy)
            print()

            if compressed_accuracy <= accuracy:
                break

    prompt_performances[candidate_prompt] = [ (right_count_pos/total_pos) * 0.5 + (right_count_neg/total_neg) * 0.5, (compressed_right_count_pos/total_pos) * 0.5 + (compressed_right_count_neg/total_neg) * 0.5 ]
    print()


In [ ]:
accuracy = (right_count_pos/total_pos) * 0.5 + (right_count_neg/total_neg) * 0.5
compressed_accuracy = (compressed_right_count_pos/total_pos) * 0.5 + (compressed_right_count_neg/total_neg) * 0.5
print('Accuracy: ', accuracy)
print('Compressed Accuracy: ', compressed_accuracy)
